In [1]:
import pandas as pd
import altair as alt

In [13]:
df = pd.read_csv('rotterdam_weather_org_17-22.csv')
df.loc[df.RH == -1, 'RH'] = 0
df.head()

,YYYYMMDD,H,FF,T,RH,M,R,S,O,Y
0,20170101,1,40,20,0,1,1,0,0,0
1,20170101,2,40,19,0,1,0,0,0,0
2,20170101,3,40,16,0,0,0,0,0,0
3,20170101,4,50,18,0,0,0,0,0,0
4,20170101,5,50,11,0,0,0,0,0,0


In [14]:
dag_neerslag = alt.Chart(df).mark_bar().encode(x='YYYYMMDD', y='sum(RH)').transformed_data()
dag_temp_min = alt.Chart(df).mark_bar().encode(x='YYYYMMDD', y='min(T)').transformed_data()
dag_temp_max = alt.Chart(df).mark_bar().encode(x='YYYYMMDD', y='max(T)').transformed_data()
dag_wind_mean = alt.Chart(df).mark_bar().encode(x='YYYYMMDD', y='mean(FF)').transformed_data()
dag_type_regen = alt.Chart(df).mark_bar().encode(x='YYYYMMDD', y='sum(R)').transformed_data()
dag_type_mist = alt.Chart(df).mark_bar().encode(x='YYYYMMDD', y='sum(M)').transformed_data()
dag_type_sneeuw = alt.Chart(df).mark_bar().encode(x='YYYYMMDD', y='sum(S)').transformed_data()
dag_type_onweer = alt.Chart(df).mark_bar().encode(x='YYYYMMDD', y='sum(O)').transformed_data()
dag_type_ijs = alt.Chart(df).mark_bar().encode(x='YYYYMMDD', y='sum(Y)').transformed_data()

In [17]:
df_comb = pd.concat((
    pd.to_datetime(dag_neerslag['YYYYMMDD'], format='%Y%m%d'), 
    dag_neerslag[['sum_RH']] / 10,
    dag_temp_max[['max_T']] / 10,
    dag_temp_min[['min_T']] / 10,
    dag_wind_mean[['mean_FF']] / 10
), axis=1)
df_comb.columns = ['date', 'precipitation', 'temp_max', 'temp_min', 'wind']
df_comb['weather'] = 'sun'
df_comb.loc[dag_type_regen.sum_R > 0, 'weather'] = 'rain'
df_comb.loc[dag_type_mist.sum_M > 0, 'weather'] = 'fog'
df_comb.loc[dag_type_sneeuw.sum_S > 0, 'weather'] = 'snow'
#df_comb.loc[dag_type_onweer.sum_O > 0, 'weather'] = 'lightning'
df_comb.loc[dag_type_ijs.sum_Y > 0, 'weather'] = 'ice'
df_comb = df_comb[df_comb.date >= '2019-01-01']
#df_comb.head()

In [18]:
df_comb.to_csv('rotterdam_weather_19-22.csv', index=False)